In [8]:
import os 
from pathlib import Path
import json
from openai import OpenAI

In [9]:
current_dir = Path.cwd()
DATASETS_LOCATION = os.path.join(current_dir.parent.parent.parent, 'datasets')
MODELS_LOCATION = os.path.join(current_dir.parent.parent, 'models')
CONFIG_LOCATION = os.path.join(current_dir.parent.parent, 'config.json')

In [12]:
with open(CONFIG_LOCATION, 'r') as configs:
    config = json.load(configs)

samba_cloud_api_key = config['samba_cloud_api_key']
model = 'Meta-Llama-3.3-70B-Instruct'
client = OpenAI(
    api_key=samba_cloud_api_key,
    base_url="https://api.sambanova.ai/v1/"
)

In [36]:
texto_prueba = """
Blood Test Results and Analysis
Patient Name: John Doe
Date of Analysis: January 5, 2025
Doctor: Dr. Emily Carter
Lab: HealthCare Diagnostics

Summary of Results
The following is an analysis of the patient's recent blood test results, with a detailed explanation of each parameter.

Glucose: 0.7395967125241718 mmol/L
The patient's glucose level is within the healthy range, indicating stable blood sugar metabolism.

Hemoglobin: 0.7136309861450383 g/dL
Hemoglobin levels are appropriate, supporting adequate oxygen transport in the blood.

Platelets: 0.8684912414028263 x10⁹/L
Platelet count is well within the normal range, which supports proper clotting function.

White Blood Cells (WBC): 0.6874330284922628 x10⁹/L
WBC count is healthy, indicating a robust immune system.

Red Blood Cells (RBC): 0.5298953992757882 x10¹²/L
RBC levels are slightly lower than average, which could suggest mild anemia. Monitoring is recommended.

Hematocrit: 0.2900059089747371
The hematocrit level is slightly below normal, potentially indicating hydration issues or anemia.

Mean Corpuscular Volume (MCV): 0.6310450180806368 fL
MCV is within the normal range, which is a positive indicator of consistent red blood cell size.

Mean Corpuscular Hemoglobin: 0.0013278578317175 pg
This value is exceedingly low but likely an outlier or error, as it does not align with clinical significance.

Mean Corpuscular Hemoglobin Concentration (MCHC): 0.7958288704767718
MCHC is optimal, reflecting a healthy concentration of hemoglobin in red blood cells.

Insulin: 0.0341291220877673 U/L
Insulin levels are low, which is appropriate for a fasting state and indicates no insulin resistance.

Body Mass Index (BMI): 0.0717741989094826
BMI is in a healthy range, correlating with good overall body composition.

Systolic Blood Pressure (SBP): 0.1855955968893292 mmHg
Systolic pressure is normal, showing no signs of hypertension.

Diastolic Blood Pressure (DBP): 0.0714546096693165 mmHg
Diastolic pressure is well-controlled, supporting cardiovascular health.

Triglycerides: 0.6534723763050316 mmol/L
Triglyceride levels are within the ideal range, indicating low cardiovascular risk.

HbA1c: 0.5026647785611607%
This value is consistent with excellent blood sugar control over the past three months.

LDL Cholesterol: 0.2155602381567172 mmol/L
LDL cholesterol, often referred to as "bad cholesterol," is low, which is beneficial for cardiovascular health.

HDL Cholesterol: 0.5129405631422954 mmol/L
HDL cholesterol, known as "good cholesterol," is at a healthy level.

ALT (Alanine Aminotransferase): 0.0641873469615352 U/L
Liver function is normal, with ALT levels indicating no liver stress or damage.

AST (Aspartate Aminotransferase): 0.610826509528389 U/L
AST levels are also within the normal range, further confirming liver health.

Heart Rate: 0.9394848536044538 bpm
Heart rate is slightly elevated but not concerning unless symptoms such as dizziness or fatigue are present.

Creatinine: 0.0955115282493801 mg/dL
Kidney function appears normal, with no signs of impaired filtration.

Troponin: 0.4659569674775698 ng/mL
Troponin levels are normal, with no indication of cardiac muscle damage.

C-reactive Protein (CRP): 0.7692300746279673 mg/L
CRP levels are low, indicating minimal inflammation in the body.

Overall Assessment
The blood test results indicate that the patient is in good health with no significant abnormalities. Minor findings, such as slightly low red blood cell count and hematocrit levels, should be monitored over time. Maintaining a balanced diet, staying hydrated, and following up with routine check-ups will ensure continued health and well-being.

Doctor's Notes:
The patient is currently classified as Healthy. Regular exercise, a balanced diet, and periodic health monitoring are recommended to sustain optimal health.
"""

In [37]:
system_prompt = """
You are values detecter. you will be given a text with the results of a blood analysis. Your job is to find 
the values for the following metrics:
You will retrieve a json file structured as follows:
blood_test_results = { "Glucose": None, "Cholesterol": None, "Hemoglobin": None, "Platelets": None, "White Blood Cells": None, "Red Blood Cells": None, "Hematocrit": None, "Mean Corpuscular Volume": None, "Mean Corpuscular Hemoglobin": None, "Mean Corpuscular Hemoglobin Concentration": None, "Insulin": None, "BMI": None, "Systolic Blood Pressure": None, "Diastolic Blood Pressure": None, "Triglycerides": None, "HbA1c": None, "LDL Cholesterol": None,29 "HDL Cholesterol": None, "ALT": None, "AST": None, "Heart Rate": None, "Creatinine": None, "Troponin": None, "C-reactive Protein": None}
Where the value of each key is None if it is not present in the text, the value otherwise. 
Do not round the values. The final output must be a valid json file. 
Do not add any comment or observation. Just retrieve the json file, it will be used by a python script with 
json.loads.
"""

In [40]:
completion = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": texto_prueba}
    ],
    stream=False
)

response = completion.choices[0].message.content.lstrip('`').rstrip('`')

In [41]:
json.loads(response)

{'Glucose': 0.7395967125241718,
 'Cholesterol': None,
 'Hemoglobin': 0.7136309861450383,
 'Platelets': 0.8684912414028263,
 'White Blood Cells': 0.6874330284922628,
 'Red Blood Cells': 0.5298953992757882,
 'Hematocrit': 0.2900059089747371,
 'Mean Corpuscular Volume': 0.6310450180806368,
 'Mean Corpuscular Hemoglobin': 0.0013278578317175,
 'Mean Corpuscular Hemoglobin Concentration': 0.7958288704767718,
 'Insulin': 0.0341291220877673,
 'BMI': 0.0717741989094826,
 'Systolic Blood Pressure': 0.1855955968893292,
 'Diastolic Blood Pressure': 0.0714546096693165,
 'Triglycerides': 0.6534723763050316,
 'HbA1c': 0.5026647785611607,
 'LDL Cholesterol': 0.2155602381567172,
 'HDL Cholesterol': 0.5129405631422954,
 'ALT': 0.0641873469615352,
 'AST': 0.610826509528389,
 'Heart Rate': 0.9394848536044538,
 'Creatinine': 0.0955115282493801,
 'Troponin': 0.4659569674775698,
 'C-reactive Protein': 0.7692300746279673}